In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
def get_df(ty='clicks'):
    if ty=='clicks':
        for i in range(no_chunks):
            for j in range(5):
                files = glob.glob('../00_submissions/' + outputfolder + '/sub_' + str(i) + '_' + ty + '_' + str(j) + '*.parquet')
                print(files)
                if i == 0 and j==0:
                    df = cudf.concat([cudf.read_parquet(x) for x in files])
                    df['score'] = df[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                    df.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                    print(df.isna().sum().sum())
                else:
                    df_tmp = cudf.concat([cudf.read_parquet(x) for x in files])
                    df_tmp['score'] = df_tmp[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                    df_tmp.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                    df_tmp = df_tmp.rename(columns={'score':'score_2'})
                    print(df_tmp.isna().sum().sum())
                    df = df.merge(
                        df_tmp,
                        how='left',
                        on=['session', 'cand']
                    )
                    print(df.isna().sum())
                    df['score'] = df['score'].fillna(0) + df['score_2'].fillna(0)
                    df.drop(['score_2'], inplace=True, axis=1)
                    del df_tmp
                    gc.collect()
                print(df.shape)
    elif ty=='carts':
        for i in range(no_chunks):
            for j in range(2):
                files = glob.glob('../00_submissions/' + outputfolder + '/sub_' + str(i) + '_' + ty + '_' + str(j) + '*.parquet')
                print(files)
                if i == 0 and j==0:
                    df = cudf.concat([cudf.read_parquet(x) for x in files])
                    df['score'] = df[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                    df.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                    print(df.isna().sum().sum())
                else:
                    df_tmp = cudf.concat([cudf.read_parquet(x) for x in files])
                    df_tmp['score'] = df_tmp[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                    df_tmp.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                    df_tmp = df_tmp.rename(columns={'score':'score_2'})
                    print(df_tmp.isna().sum().sum())
                    df = df.merge(
                        df_tmp,
                        how='left',
                        on=['session', 'cand']
                    )
                    print(df.isna().sum())
                    df['score'] = df['score'].fillna(0) + df['score_2'].fillna(0)
                    df.drop(['score_2'], inplace=True, axis=1)
                    del df_tmp
                    gc.collect()
                print(df.shape)
    else:
        for i in range(no_chunks):
            files = glob.glob('../00_submissions/' + outputfolder + '/sub_' + str(i) + '_' + ty + '_' + str(0) + '*.parquet')
            if i == 0:
                df = cudf.concat([cudf.read_parquet(x) for x in files])
                print(df.head())
                #df['score'] = df[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                #df.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                print(df.isna().sum().sum())
            else:
                df_tmp = cudf.concat([cudf.read_parquet(x) for x in files])
                print(df_tmp.head())
                #df_tmp['score'] = df_tmp[['score_' + str(iseed) for iseed in range(no_seeds)]].sum(axis=1)
                #df_tmp.drop(['score_' + str(iseed) for iseed in range(no_seeds)], axis=1, inplace=True)
                df_tmp = df_tmp.rename(columns={'score':'score_2'})
                print(df_tmp.isna().sum().sum())
                df = df.merge(
                    df_tmp,
                    how='left',
                    on=['session', 'cand']
                )
                print(df.isna().sum())
                df['score'] = df['score'].fillna(0) + df['score_2'].fillna(0)
                df.drop(['score_2'], inplace=True, axis=1)
                del df_tmp
                gc.collect()
            print(df.shape)
    return(df)

def prep_xgb(df, df_baseline, ty):
    df = df.sort_values(['session', 'score'], ascending=[True, False])
    df['dummy'] = 1
    df['rank'] = df.groupby(['session']).dummy.cumsum()
    df.drop(['dummy'], axis=1, inplace=True)
    df = df[df['rank']<21]
    
    df = df[['session', 'cand']].groupby(['session']).agg(list).reset_index()
    df = df.to_pandas()
    print(df.shape, df_baseline.shape)
    
    df = df.merge(
        df_baseline,
        how='left',
        on=['session']
    )
    
    list1 = df['cand'].tolist()
    list2 = df['labels'].tolist()
    
    out = []
    for i, (l1, l2) in enumerate(zip(list1, list2)):
        l1 = l1.tolist()
        if len(l1)==20:
            out.append(l1[0:20])
        else:
            l1 = l1 + [x for x in l2 if x not in l1]
            l1 = l1[0:20]
            out.append(l1)
    
    df_final = df[['session']]
    df_final['labels'] = out
    df_final['labels'] = df_final['labels'].apply(lambda x: " ".join(map(str,x)))
    df_final['session_type'] = df_final['session'].astype('str') + '_' + ty
    return(df_final)

In [4]:
import cudf
import glob
import gc

import pandas as pd

no_chunks = 5
no_seeds = 3
outputfolder = '24_XGB_Rerun_RedruceCandidate_DifferentWeights_Folds_ChrisCo_SameDay_v3_emb_mt'

In [6]:
# Submission file from https://www.kaggle.com/code/cdeotte/candidate-rerank-model-lb-0-575 
# to fill up sessions with less than 20 Candidates
df_baseline = pd.read_csv('../00_submissions/' + '01_baseline' + '/submission.csv')
df_baseline['session'] = df_baseline.session_type.apply(lambda x: int(x.split('_')[0]))
df_baseline['type'] = df_baseline.session_type.apply(lambda x: x.split('_')[1])
df_baseline['session'].drop_duplicates().shape

In [9]:
%%time

df_baseline.labels = df_baseline.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

CPU times: user 51.5 s, sys: 2.87 s, total: 54.3 s
Wall time: 54.3 s


In [10]:
# %%time
ty='clicks'
df = get_df(ty=ty)
df_clicks_final = prep_xgb(df, df_baseline[df_baseline['type']==ty], ty=ty)
del df
gc.collect()

In [15]:
df_clicks_final[['session_type', 'labels']].to_csv('../00_submissions/' + outputfolder + '/submission_2.csv', index=False)